<a href="https://colab.research.google.com/github/i-m-himanshu/BERT-vs-GAN-BERT/blob/main/BERT/BERT_emotion_classification4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

#Import Data

In [ ]:
data_train = pd.read_csv('data_train.csv', encoding='utf-8')
data_test = pd.read_csv('data_test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral'] #Predefined classes

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 3867
size of validation set: 1300
sadness    1083
neutral    1081
joy        1080
anger       990
fear        933
Name: Emotion, dtype: int64


,Emotion,Text
0,sadness,But why does Mr Ross always look sad ?
1,fear,Before an examination: I had the fealing that ...
2,anger,The trick was really intolerable. They hid it...
3,anger,"It joined me for the rest of the holiday , muc..."
4,neutral,Why not ?
5,sadness,"I know , it's not as I had planned ."
6,neutral,Tony Bush .
7,neutral,Freeze !
8,neutral,Visa .
9,anger,You didn't ring me last night.You said you wou...


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

#Data Preprocessing

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


#Training and Validation Split

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc) #Loading the pretrained BERT for text classification

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5,5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
645/645 [==============================] - 409s 599ms/step - loss: 1.2004 - accuracy: 0.5198 - val_loss: 0.9166 - val_accuracy: 0.6908
Epoch 2/5
645/645 [==============================] - 383s 594ms/step - loss: 0.6494 - accuracy: 0.7874 - val_loss: 0.8763 - val_accuracy: 0.7223
Epoch 3/5
645/645 [==============================] - 383s 593ms/step - loss: 0.4658 - accuracy: 0.8588 - val_loss: 0.9308 - val_accuracy: 0.7338
Epoch 4/5
645/645 [==============================] - 383s 594ms/step - loss: 0.2862 - accuracy: 0.9209 - val_loss: 0.9454 - val_accuracy: 0.7562
Epoch 5/5
645/645 [==============================] - 383s 594ms/step - loss: 0.1910 - accuracy: 0.9413 - val_loss: 0.9925 - val_accuracy: 0.7477


#Making Predictions

Generally, the term “validation set” is used interchangeably with the term “test set” and refers to a sample of the dataset held back from training the model.

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.76      0.70      0.73       274
     sadness       0.69      0.70      0.69       271
        fear       0.72      0.62      0.67       240
       anger       0.65      0.68      0.66       255
     neutral       0.68      0.78      0.73       260

    accuracy                           0.70      1300
   macro avg       0.70      0.70      0.70      1300
weighted avg       0.70      0.70      0.70      1300



array([[192,  19,  11,  17,  35],
       [ 14, 189,  19,  30,  19],
       [ 16,  25, 150,  34,  15],
       [ 12,  24,  18, 173,  28],
       [ 18,  17,  10,  12, 203]])